In [1]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
from model_saver import model_saver_wandb as model_saver
import training_testing
from itertools import combinations
import gc
from sklearn.decomposition import PCA
import lightgbm as lgb


In [2]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_csv('train.csv')
train = train.query('seconds_in_bucket <= 290').reset_index(drop=True)
train.head()
train.date_id.value_counts()

date_id
480    6000
353    6000
363    6000
362    6000
360    6000
       ... 
4      5760
2      5730
1      5730
3      5730
0      5730
Name: count, Length: 481, dtype: int64

In [5]:
train.seconds_in_bucket.value_counts()

seconds_in_bucket
0      95236
10     95236
280    95236
270    95236
260    95236
250    95236
240    95236
230    95236
220    95236
210    95236
200    95236
190    95236
180    95236
170    95236
160    95236
150    95236
140    95236
130    95236
120    95236
110    95236
100    95236
90     95236
80     95236
70     95236
60     95236
50     95236
40     95236
30     95236
20     95236
290    95236
Name: count, dtype: int64

In [6]:
def imbalance_calculator(x):
    
    x_copy = x.copy()
    
    x_copy['imb_s1'] = x.eval('(bid_size - ask_size) / (bid_size + ask_size)')
    x_copy['imb_s2'] = x.eval('(imbalance_size - matched_size) / (matched_size + imbalance_size)')
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            if i>j:
                x_copy[f'{a}_{b}_imb'] = x.eval(f'({a} - {b}) / ({a} + {b} + {0.001})')
                    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    max_ = x[[a,b,c]].max(axis=1)
                    min_ = x[[a,b,c]].min(axis=1)
                    mid_ = x[[a,b,c]].sum(axis=1)-min_-max_

                    x_copy[f'{a}_{b}_{c}_imb2'] = (max_-mid_)/(mid_-min_+0.001)
    x_copy.replace([np.inf, -np.inf], 0, inplace=True)
    return x_copy



In [7]:
median_vol = pd.read_csv("archive/MedianVolV2.csv")
median_vol.index.name = "stock_id";
median_vol = median_vol[['overall_medvol', "first5min_medvol", "last5min_medvol"]]

In [8]:
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()

In [16]:
def feat_eng(df):
    
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    df = df.merge(median_vol, how = "left", left_on = "stock_id", right_index = True)
    
    df['bid_plus_ask_sizes'] = df['bid_size'] + train['ask_size']
#     df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
#     df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0) 
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')

    df['ask_x_size'] = df.eval('ask_size*ask_price')
    df['bid_x_size'] = df.eval('bid_size*bid_price')
        
    df['ask_minus_bid'] = df['ask_x_size'] - df['bid_x_size'] 
    
    df["bid_size_over_ask_size"] = df["bid_size"].div(df["ask_size"])
    df["bid_price_over_ask_price"] = df["bid_price"].div(df["ask_price"])
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for c in combinations(prices, 2):
        
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_times_{c[1]}'] = (df[f'{c[0]}'] * df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in combinations(prices, 3):
        
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
    
        
    df.drop(columns=[
        # 'date_id', 
        'reference_price_far_price_imb',
        'reference_price_minus_near_price',
        'reference_price_near_price_imb',
        'far_price_near_price_imb',
        'far_price_ask_price_imb',
        'far_price_bid_price_imb',
        'far_price_minus_wap',
        'std_size',
        'bid_size_over_ask_size',
        'ask_price_bid_price_imb',
        'near_price_times_wap'
    ], inplace=True)
        
    gc.collect()

    df.replace([np.inf, -np.inf], 0, inplace=True)
    
    return df

In [17]:
y = train['target'].values
X = feat_eng(train)

In [18]:
nulls = X.isna().sum(axis = 0)

In [19]:
prices = [c for c in train.columns if 'price' in c]
pca_prices = PCA(n_components=1)
X['pca_prices'] = pca_prices.fit_transform(X[prices].fillna(1))

In [20]:
lgbm_columns = ['stock_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'overall_medvol', 'first5min_medvol',
       'last5min_medvol', 'bid_plus_ask_sizes', 'imbalance_ratio', 'imb_s1',
       'imb_s2', 'ask_x_size', 'bid_x_size', 'ask_minus_bid',
       'bid_price_over_ask_price', 'reference_price_minus_far_price',
       'reference_price_times_far_price', 'reference_price_times_near_price',
       'reference_price_minus_ask_price', 'reference_price_times_ask_price',
       'reference_price_ask_price_imb', 'reference_price_minus_bid_price',
       'reference_price_times_bid_price', 'reference_price_bid_price_imb',
       'reference_price_minus_wap', 'reference_price_times_wap',
       'reference_price_wap_imb', 'far_price_minus_near_price',
       'far_price_times_near_price', 'far_price_minus_ask_price',
       'far_price_times_ask_price', 'far_price_minus_bid_price',
       'far_price_times_bid_price', 'far_price_times_wap', 'far_price_wap_imb',
       'near_price_minus_ask_price', 'near_price_times_ask_price',
       'near_price_ask_price_imb', 'near_price_minus_bid_price',
       'near_price_times_bid_price', 'near_price_bid_price_imb',
       'near_price_minus_wap', 'near_price_wap_imb',
       'ask_price_minus_bid_price', 'ask_price_times_bid_price',
       'ask_price_minus_wap', 'ask_price_times_wap', 'ask_price_wap_imb',
       'bid_price_minus_wap', 'bid_price_times_wap', 'bid_price_wap_imb',
       'reference_price_far_price_near_price_imb2',
       'reference_price_far_price_ask_price_imb2',
       'reference_price_far_price_bid_price_imb2',
       'reference_price_far_price_wap_imb2',
       'reference_price_near_price_ask_price_imb2',
       'reference_price_near_price_bid_price_imb2',
       'reference_price_near_price_wap_imb2',
       'reference_price_ask_price_bid_price_imb2',
       'reference_price_ask_price_wap_imb2',
       'reference_price_bid_price_wap_imb2',
       'far_price_near_price_ask_price_imb2',
       'far_price_near_price_bid_price_imb2', 'far_price_near_price_wap_imb2',
       'far_price_ask_price_bid_price_imb2', 'far_price_ask_price_wap_imb2',
       'far_price_bid_price_wap_imb2', 'near_price_ask_price_bid_price_imb2',
       'near_price_ask_price_wap_imb2', 'near_price_bid_price_wap_imb2',
       'ask_price_bid_price_wap_imb2', 'pca_prices']

In [21]:
lgbm = lgb.Booster(model_file='lgbm_model.lgb')
lgbm_preds = lgbm.predict(X[lgbm_columns])
X['lgbm_preds'] = lgbm_preds

In [24]:
X

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,far_price_near_price_wap_imb2,far_price_ask_price_bid_price_imb2,far_price_ask_price_wap_imb2,far_price_bid_price_wap_imb2,near_price_ask_price_bid_price_imb2,near_price_ask_price_wap_imb2,near_price_bid_price_wap_imb2,ask_price_bid_price_wap_imb2,pca_prices,lgbm_preds
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,...,-1.0,-1.000214,-1.000026,-1.000188,-1.000214,-1.000026,-1.000188,0.138298,-0.000206,-0.203192
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,...,-1.0,-1.000764,-1.000660,-1.000104,-1.000764,-1.000660,-1.000104,6.346154,0.000256,1.151963
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,...,-1.0,-1.000896,-1.000298,-1.000597,-1.000896,-1.000298,-1.000597,0.499162,-0.000430,-2.417485
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,...,-1.0,-1.000215,-1.000214,-1.000001,-1.000215,-1.000214,-1.000001,214.000000,0.000219,0.456065
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,...,-1.0,-1.000622,-1.000016,-1.000606,-1.000622,-1.000016,-1.000606,0.026403,-0.000614,-2.745228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2857075,195,480,290,6099963.33,-1,1.000317,18633850.25,NaN,NaN,1.000201,...,-1.0,-1.000116,-1.000026,-1.000090,-1.000116,-1.000026,-1.000090,0.288889,0.000479,-0.290608
2857076,196,480,290,67949.56,-1,0.999357,7936446.96,NaN,NaN,0.999871,...,-1.0,-1.000258,-1.000153,-1.000105,-1.000258,-1.000153,-1.000105,1.457143,-0.000379,0.545304
2857077,197,480,290,3772211.05,-1,0.996538,7660852.97,NaN,NaN,0.996445,...,-1.0,-1.001035,-1.000587,-1.000448,-1.001035,-1.000587,-1.000448,1.311659,-0.005513,0.302351
2857078,198,480,290,3224772.38,-1,0.999210,84569369.22,NaN,NaN,0.998729,...,-1.0,-1.000241,-1.000133,-1.000108,-1.000241,-1.000133,-1.000108,1.231481,-0.001785,0.150930


In [25]:
X.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'overall_medvol', 'first5min_medvol',
       'last5min_medvol', 'bid_plus_ask_sizes', 'imbalance_ratio', 'imb_s1',
       'imb_s2', 'ask_x_size', 'bid_x_size', 'ask_minus_bid',
       'bid_price_over_ask_price', 'reference_price_minus_far_price',
       'reference_price_times_far_price', 'reference_price_times_near_price',
       'reference_price_minus_ask_price', 'reference_price_times_ask_price',
       'reference_price_ask_price_imb', 'reference_price_minus_bid_price',
       'reference_price_times_bid_price', 'reference_price_bid_price_imb',
       'reference_price_minus_wap', 'reference_price_times_wap',
       'reference_price_wap_imb', 'far_price_minus_near_price',
       'far_price_times_near_price', 'far_price_minus_ask_price',
       

In [26]:
def zero_sum(prices, volumes):
    
#    I got this idea from https://github.com/gotoConversion/goto_conversion/
    
    std_error = np.sqrt(volumes)
    step = np.sum(prices)/np.sum(std_error)
    out = prices-std_error*step
    
    return out

In [27]:
import importlib

In [28]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(X)
hidden_size = 64
trading_data.generate_batches()

 10%|▉         | 19/200 [00:12<01:23,  2.18it/s]

Missing Targets for day=438,for stock_id=19, Excluding


 50%|█████     | 101/200 [00:59<00:45,  2.19it/s]

Missing Targets for day=328,for stock_id=101, Excluding


 66%|██████▌   | 131/200 [01:17<01:35,  1.38s/it]

...

 79%|███████▉  | 158/200 [01:29<00:18,  2.33it/s]

Missing Targets for day=388,for stock_id=158, Excluding


100%|██████████| 200/200 [01:53<00:00,  1.75it/s]


Length of train: 385, Length of test 96


100%|██████████| 95/95 [00:00<00:00, 10546.54it/s]


In [19]:
len(trading_data.train_batches[0])

191

In [20]:
for x in trading_data.train_batches:
    for t1 in x:
        for t in t1:
            if len(t)!=42:
                print(len(t))
        

In [21]:
stat_cols = ['seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'overall_medvol', 'first5min_medvol',
       'last5min_medvol', 'bid_plus_ask_sizes', 'imbalance_ratio', 'imb_s1',
       'imb_s2', 'ask_x_size', 'bid_x_size', 'ask_minus_bid',
       'bid_price_over_ask_price', 'reference_price_minus_ask_price',
       'reference_price_times_ask_price', 'reference_price_ask_price_imb',
       'reference_price_minus_bid_price', 'reference_price_times_bid_price',
       'reference_price_bid_price_imb', 'reference_price_minus_wap',
       'reference_price_times_wap', 'reference_price_wap_imb',
       'ask_price_minus_bid_price', 'ask_price_times_bid_price',
       'ask_price_minus_wap', 'ask_price_times_wap', 'ask_price_wap_imb',
       'bid_price_minus_wap', 'bid_price_times_wap', 'bid_price_wap_imb',
       'reference_price_ask_price_bid_price_imb2',
       'reference_price_ask_price_wap_imb2',
       'reference_price_bid_price_wap_imb2', 'ask_price_bid_price_wap_imb2']

In [22]:
X['stats'] = pd.Series(X[stat_cols].fillna(-1).values.tolist())

In [29]:
importlib.reload(torch_classes)
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [34]:
def model_pipeline(trading_df:torch_classes.TradingData, config:dict):
    with wandb.init(project="Optviver", config=config,save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        model = torch_classes.GRUNetV2(81,config['hidden_size'], num_layers=2).to('cuda:0')
        print(model)
        config = wandb.config
        optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])
        trading_df.reset_hidden(config['hidden_size'], config['num_layers'])
        criterion = nn.SmoothL1Loss()
        
        training_testing.train_model(trading_df,model,config,optimizer,criterion)

    return(model)



In [31]:
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [32]:
config_static = {'learning_rate':0.00001, 'hidden_size':16, 'num_layers':2}

In [35]:
model = model_pipeline(trading_data, config_static)

GRUNetV2(
  (gru): GRU(81, 16, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (batch_norm): BatchNorm1d(81, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  (fc0): Linear(in_features=16, out_features=16, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=16, out_features=1, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
)


  0%|          | 0/10000 [00:00<?, ?it/s]

   stock  day  time    target      pred
0      0    0     0 -2.340078  0.056889
1      1    0     0 -8.130074  0.202185
2      2    0     0  0.790358  0.246280
3      3    0     0  6.059408  0.037527
4      4    0     0  0.499487  0.205801
5      5    0     0  5.190372  0.166963
6      6    0     0 -4.439950  0.237778
7      7    0     0 -3.280044  0.163919
8      8    0     0  0.419617  0.099438
9      9    0     0  1.959801  0.051684
created path
   stock  day  time    target      pred
0      0    0     0 -2.340078 -0.670580
1      1    0     0 -8.130074 -0.084390
2      2    0     0  0.790358  0.331256
3      3    0     0  6.059408 -0.416079
4      4    0     0  0.499487  0.141223
5      5    0     0  5.190372 -0.004307
6      6    0     0 -4.439950  0.068711
7      7    0     0 -3.280044 -0.089035
8      8    0     0  0.419617 -0.396527
9      9    0     0  1.959801 -0.213220
   stock  day  time    target      pred
0      0    0     0 -2.340078 -1.227026
1      1    0     0 -8.1300

wandb: Network error (ConnectionError), entering retry loop.


   stock  day  time    target      pred
0      0    0     0 -2.340078 -1.477862
1      1    0     0 -8.130074 -1.659792
2      2    0     0  0.790358 -0.784501
3      3    0     0  6.059408 -1.224919
4      4    0     0  0.499487 -0.243962
5      5    0     0  5.190372 -2.675981
6      6    0     0 -4.439950 -1.151475
7      7    0     0 -3.280044 -2.784948
8      8    0     0  0.419617 -0.137615
9      9    0     0  1.959801  0.381998


...

wandb: Network error (ConnectionError), entering retry loop.


   stock  day  time    target      pred
0      0    0     0 -2.340078 -0.785306
1      1    0     0 -8.130074 -2.571899
2      2    0     0  0.790358 -0.763124
3      3    0     0  6.059408 -0.764036
4      4    0     0  0.499487 -0.658016
5      5    0     0  5.190372 -2.720518
6      6    0     0 -4.439950 -0.805771
7      7    0     0 -3.280044 -3.708956
8      8    0     0  0.419617  0.452716
9      9    0     0  1.959801  0.458487


wandb: Network error (ConnectionError), entering retry loop.


   stock  day  time    target      pred
0      0    0     0 -2.340078 -0.774358
1      1    0     0 -8.130074 -2.679548
2      2    0     0  0.790358 -0.761382
3      3    0     0  6.059408 -0.761852
4      4    0     0  0.499487 -0.759583
5      5    0     0  5.190372 -2.686026
6      6    0     0 -4.439950 -0.788823
7      7    0     0 -3.280044 -3.700814
8      8    0     0  0.419617  0.417264
9      9    0     0  1.959801  0.426779


wandb: Network error (ConnectionError), entering retry loop.
Traceback (most recent call last):
  File "C:\Users\Nick\AppData\Local\Temp\ipykernel_9876\726279498.py", line 12, in model_pipeline
    training_testing.train_model(trading_df,model,config,optimizer,criterion)
  File "c:\Users\Nick\Documents\GitHub\OptiverKaggle\training_testing.py", line 36, in train_model
    hidden_in = torch.stack([x.hidden for x in stocks]).transpose(0,1)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


In [ ]:
torch.cuda.empty_cache()

In [ ]:
trading_data.reset_hidden(config_static['hidden_size'], config_static['num_layers'])